In [ ]:
!pip install transformers datasets gradio pyngrok

In [ ]:
import gradio as gr
from datasets import load_dataset
from transformers import BartForConditionalGeneration, BartTokenizer, pipeline

In [ ]:
# 데이터셋 로드 (CNN_dailymail 1000개 샘플 데이터 사용)
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1000]")
print(dataset[0])  # 첫 번째 데이터 확인

In [ ]:
# BART 모델과 토크나이저 로드
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# 요약 함수 정의
def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=130, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# 테스트: 첫 번째 뉴스 기사 요약
example_text0 = dataset[0]["article"]
print("Original Article0:", example_text0)
print("Summary:", summarize(example_text0))


In [ ]:
# 마지막 뉴스 기사 요약
example_text999 = dataset[999]["article"]
print("Original Article999:", example_text999)
print("Summary:", summarize(example_text999))


In [ ]:
# DistilBERT 기반 질의응답 파이프라인 로드
qa_model = pipeline("question-answering", model="distilbert-base-uncased", tokenizer="distilbert-base-uncased")

In [ ]:
# 질의응답 함수 정의
def answer_question(context, question):
    result = qa_model(question=question, context=context)
    return result["answer"]


In [ ]:
# 테스트: 요약문 기반 질문 응답1 (메인 주제 질문)
question = "What is the main topic of the article?"
context = summarize(example_text0)
print("Question:", question)
print("Answer:", answer_question(context, question))


In [ ]:
# 테스트: 요약문 기반 질문 응답 (인물에 대한 질문)
question = "Who is the main person mentioned in the article?"
context = summarize(example_text0)
print("Question:", question)
print("Answer:", answer_question(context, question))


### 질문과 상관없이 동일한 답을 내는 문제점을 발견
원인과 해결 방안
- 1. 요약 내용이 너무 간단한 경우 -> answer_question 함수를 fine-tuning해 요악함수를 정확하게 만들기.
- 2. summarize 함수의 개선 -> summarize 함수의 출력 길이 늘리기.
- 3. 질문에 맞는 답을 하도록 질문 -> 특정한 질문에 맞게 학습시키기.

### Answer_question 함수를 fine-tuning.
1. SQuAD 데이터 셋으로 모델 학습시키기
question-answering 모델을 질문-답변(QnA) 작업에 적합하고, QnA 모델의 성능을 개선하기위해 SQuAD데이터 셋으로 질문에 대한 답변 정확도 높히기.

In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# 데이터셋 불러오기
dataset = load_dataset('squad')

# 모델과 토크나이저 준비
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 데이터셋을 모델에 맞게 준비
def preprocess_function(examples):
    # 토큰화 및 입력 길이 설정
    tokenized_examples = tokenizer(examples['question'], examples['context'], truncation=True, padding='max_length', max_length=512)

    # 시작 위치와 끝 위치 계산
    start_positions = []
    end_positions = []

    for i in range(len(examples['answers'])):
        # 정답의 첫 번째 텍스트 선택
        answer = examples['answers'][i]['text'][0]  # 리스트에서 첫 번째 텍스트를 선택

        # 각 문맥에 대해서 정답이 있는지 확인 (예시: 'context'는 리스트일 수 있음)
        context = examples['context'][i]  # 각 문맥을 가져옴

        # context 내에서 정답의 시작과 끝 위치를 찾기
        start_pos = context.find(answer)
        end_pos = start_pos + len(answer) - 1

        # 만약 정답이 문맥에서 발견되지 않으면 (예를 들어, 잘못된 데이터가 있는 경우) -1로 설정
        if start_pos == -1:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_positions.append(start_pos)
            end_positions.append(end_pos)

    tokenized_examples['start_positions'] = start_positions
    tokenized_examples['end_positions'] = end_positions

    return tokenized_examples

train_dataset = dataset['train'].map(preprocess_function, batched=True)
eval_dataset = dataset['validation'].map(preprocess_function, batched=True)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    disable_tqdm=False,  # TQDM 활성화
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Fine-tuning 시작
trainer.train()


In [ ]:
from transformers import pipeline

# fine-tuning된 모델 로드
qa_model = BertForQuestionAnswering.from_pretrained('./results')

# QnA 파이프라인 설정
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=tokenizer)

# 질문 및 문맥
context = "Harry Potter star Daniel Radcliffe turns 18 and gains access to a £20 million fortune."
question = "What is Daniel Radcliffe's fortune?"

# 답변 추출
answer = qa_pipeline(question=question, context=context)
print(f"Answer: {answer['answer']}")



In [ ]:
# 모델과 토크나이저 로드
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
qa_model = pipeline("question-answering", model="distilbert-base-uncased", tokenizer="distilbert-base-uncased")

# 요약 함수
def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=130, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 질문에 대한 답변 함수
def answer_question(context, question):
    result = qa_model(question=question, context=context)
    return result["answer"]

# Gradio 인터페이스 생성
def process(text, question):
    summary = summarize(text)  # 텍스트 요약
    answer = answer_question(summary, question)  # 요약된 텍스트에 대한 질문 응답
    return summary, answer  # 두 가지 결과 반환

# Gradio 인터페이스 설정
iface = gr.Interface(fn=process,
                     inputs=["text", "text"],  # 두 개의 입력: 기사 텍스트, 질문 텍스트
                     outputs=["text", "text"],  # 두 개의 출력: 요약, 답변
                     live=True)

iface.launch()



시도1) 한국어버젼

In [ ]:
# 공개된 한국어 BERT 모델 사용
model_name = "kykim/bert-kor-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# 한국어 텍스트 요약 함수
def summarize_korean(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    # 요약을 위해 모델을 적합하게 변경할 필요가 있을 수 있음
    summary_ids = model.generate(inputs.input_ids, max_length=130, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Gradio 인터페이스 구성 (한국어)
import gradio as gr

def process(text, question):
    summary = summarize_korean(text)  # 한국어 텍스트 요약
    answer = answer_question(summary, question)  # 요약된 텍스트에 대한 질문 응답
    return summary, answer  # 두 가지 결과 반환

iface = gr.Interface(fn=process,
                     inputs=["text", "text"],
                     outputs=["text", "text"],
                     live=True)

iface.launch()

